Code chính

In [1]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock
import talib as ta
import seaborn as sns
import matplotlib.pyplot as plt

/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def calculate_first_mondays(dates):
    if not isinstance(dates, pd.DatetimeIndex):
        dates = pd.DatetimeIndex(dates)
    dates_series = pd.Series(dates, index=dates)
    mondays = dates_series[dates_series.dt.dayofweek == 0]
    first_mondays = mondays.groupby([mondays.dt.year, mondays.dt.month]).first()
    return set(first_mondays)

In [5]:
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 50  # Initialize the investment fund 
    
    RSI_PERIOD = 14
    RSI_OVERSOLD = 30
    RSI_OVERBOUGHT = 70
    MACD_FAST = 12
    MACD_SLOW = 26
    MACD_SIGNAL = 9

    def init(self):
        close = self.data.Close
        # Calculate RSI
        self.rsi = self.I(ta.RSI, close, self.RSI_PERIOD) 
        # Calculate MACD
        macd, signal_line, _ = ta.MACD(close, self.MACD_FAST, self.MACD_SLOW, self.MACD_SIGNAL)  
        self.macd = self.I(pd.Series, macd)
        self.signal_line = self.I(pd.Series, signal_line)
        self.previous_macd = self.I(pd.Series(macd).shift, 1)
        self.previous_signal_line = self.I(pd.Series(signal_line).shift, 1)

        self.first_mondays = calculate_first_mondays(self.data.index)

    def next(self):
        # Update the fund at the start of each month
        today = self.data.index[-1]
        if today in self.first_mondays:
            self.fund += self.average_monthly_income_vnd * self.investment_percentage
            
        # Check for buy signal: RSI cross above 30 and MACD cross above Signal line
        if (self.previous_macd[-1] < self.previous_signal_line[-1] and
            self.macd[-1] >= self.signal_line[-1] and
            self.rsi[-1] > self.RSI_OVERSOLD):  
            share_price = self.data.Close[-1]
            max_shares_to_buy = self.fund // share_price
            lots_to_buy = max_shares_to_buy // 100
            
            if lots_to_buy > 0:
                shares_to_buy = lots_to_buy * 100
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                #print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")
                

def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
    stock_data = stock_data.dropna()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats,heatmap = bt.optimize(
        MACD_FAST= range(8,15,1),
        MACD_SLOW = range(10,30,1),
        MACD_SIGNAL = range(5,10,1),
        maximize = 'Buy & Hold Return [%]',
        return_heatmap = True
    )

    
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print(stats)
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print(stats["_strategy"])
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['REE']

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-26 23:06:01,583 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 700 configurations.
  output = _optimize_grid()
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/4 [00:00<?, ?it/s]

Results for REE:
Start                     2019-01-02 00:00:00
End                       2024-01-03 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                   98.081535
Equity Final [$]                  11412.15204
Equity Peak [$]                  13388.536496
Return [%]                           14.12152
Buy & Hold Return [%]               168.71494
Return (Ann.) [%]                    2.696597
Volatility (Ann.) [%]                9.233886
Sharpe Ratio                         0.292033
Sortino Ratio                        0.424233
Calmar Ratio                         0.148976
Max. Drawdown [%]                  -18.100929
Avg. Drawdown [%]                   -1.407562
Max. Drawdown Duration      565 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                   21
Win Rate [%]                        71.428571
Best Trade [%]                     149.697429
Worst Trade [%]                    -17.810097
Avg. Trade [%]   

CODE CHÍNH

In [4]:
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 50  # Initialize the investment fund 

    def init(self):
        close = self.data.Close
        # Calculate RSI
        self.rsi = self.I(ta.RSI, close, timeperiod=14) 
        # Calculate MACD
        macd, signal_line, _ = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)  
        self.macd = self.I(pd.Series, macd)
        self.signal_line = self.I(pd.Series, signal_line)
        self.previous_macd = self.I(pd.Series(macd).shift, 1)
        self.previous_signal_line = self.I(pd.Series(signal_line).shift, 1)

        self.first_mondays = calculate_first_mondays(self.data.index)

    def next(self):
        # Update the fund at the start of each month
        today = self.data.index[-1]
        if today in self.first_mondays:
            self.fund += self.average_monthly_income_vnd * self.investment_percentage
            
        # Check for buy signal: RSI cross above 30 and MACD cross above Signal line
        if (self.previous_macd[-1] < self.previous_signal_line[-1] and
            self.macd[-1] >= self.signal_line[-1] and
            self.rsi[-1] > 30):  
            share_price = self.data.Close[-1]
            max_shares_to_buy = self.fund // share_price
            lots_to_buy = max_shares_to_buy // 100
            
            if lots_to_buy > 0:
                shares_to_buy = lots_to_buy * 100
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                #print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")
                
        #if (self.rsi[-1] < RSI_OVERBOUGHT and
            #self.previous_macd[-1] > self.previous_signal_line[-1] and
            #self.macd[-1] <= self.signal_line[-1]):
            #if self.position:
                #self.position.close()


def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
    stock_data = stock_data.dropna()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    #bt.plot(filename=f'{stock_symbol}')
    
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['FPT', 'REE']

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-26 23:03:04,433 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-08-26 23:03:05,089 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for FPT:
Total investment: 2635.870604204091
Current Shares: 1800
Current Equity: 6153.669064748201
RoR: 133.45869311389512
--------------------------------------------------
Results for REE:
Total investment: 2933.768418246696
Current Shares: 2100
Current Equity: 4255.251798561151
RoR: 45.04388867558304
--------------------------------------------------


DƯỚI NÀY LÀ TEST THÔI

In [11]:
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 0  # Initialize the investment fund

    def init(self):
        close = self.data.Close
        # Calculate RSI
        self.rsi = self.I(ta.RSI, close, timeperiod=14) 
        # Calculate MACD
        macd, signal_line, _ = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)  
        self.macd = self.I(pd.Series, macd)
        self.signal_line = self.I(pd.Series, signal_line)
        self.previous_macd = self.I(pd.Series(macd).shift, 1)
        self.previous_signal_line = self.I(pd.Series(signal_line).shift, 1)
        self.first_mondays = calculate_first_mondays(self.data.index)

    def next(self):
        today = self.data.index[-1]
        if today in self.first_mondays:
            self.fund += self.average_monthly_income_vnd * self.investment_percentage
            print(f"Day: {today} total fund: {self.fund}")

        # Check for buy signal
        if (self.previous_macd[-1] < self.previous_signal_line[-1] and
            self.macd[-1] >= self.signal_line[-1] and
            self.rsi[-1] > 30):  
            
            share_price = self.data.Close[-1]
            max_shares_to_buy = self.fund // share_price
            lots_to_buy = max_shares_to_buy // 100
            
            if lots_to_buy > 0:
                shares_to_buy = lots_to_buy * 100
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")
            
            shares_to_buy = self.fund // share_price
            shares_to_buy = (shares_to_buy // 100) * 100
            if shares_to_buy > 0:
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")

        #if (self.rsi[-1] < RSI_OVERBOUGHT and
            #self.previous_macd[-1] > self.previous_signal_line[-1] and
            #self.macd[-1] <= self.signal_line[-1]):
            #if self.position:
                #self.position.close()



def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Diagnostic Check 1: Print the date range of stock data
    print("Stock data date range:", stock_data.index.min(), "to", stock_data.index.max())

    # Calculate first Mondays
    first_mondays = calculate_first_mondays(stock_data.index)
    
    # Diagnostic Check 2: Print the first Mondays
    print("First Mondays identified:", sorted(list(first_mondays)))

    # Normalize dates for merging
    stock_data.index = stock_data.index.normalize()


    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']


    # Drop rows with NA values
    stock_data = stock_data.dropna()


    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    bt.plot(filename=f'{stock_symbol}')
    print(stats)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['VCB']  # Assume more stocks listed if needed

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-14 22:28:59,982 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Stock data date range: 2019-01-02 07:00:00 to 2024-01-03 07:00:00
First Mondays identified: [Timestamp('2019-01-07 07:00:00'), Timestamp('2019-02-11 07:00:00'), Timestamp('2019-03-04 07:00:00'), Timestamp('2019-04-01 07:00:00'), Timestamp('2019-05-06 07:00:00'), Timestamp('2019-06-03 07:00:00'), Timestamp('2019-07-01 07:00:00'), Timestamp('2019-08-05 07:00:00'), Timestamp('2019-09-09 07:00:00'), Timestamp('2019-10-07 07:00:00'), Timestamp('2019-11-04 07:00:00'), Timestamp('2019-12-02 07:00:00'), Timestamp('2020-01-06 07:00:00'), Timestamp('2020-02-03 07:00:00'), Timestamp('2020-03-02 07:00:00'), Timestamp('2020-04-06 07:00:00'), Timestamp('2020-05-04 07:00:00'), Timestamp('2020-06-01 07:00:00'), Timestamp('2020-07-06 07:00:00'), Timestamp('2020-08-03 07:00:00'), Timestamp('2020-09-07 07:00:00'), Timestamp('2020-10-05 07:00:00'), Timestamp('2020-11-02 07:00:00'), Timestamp('2020-12-07 07:00:00'), Timestamp('2021-01-04 07:00:00'), Timestamp('2021-02-01 07:00:00'), Timestamp('2021-03-01 0

/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

Start                     2019-01-02 00:00:00
End                       2024-01-03 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                   91.446843
Equity Final [$]                  10771.24824
Equity Peak [$]                   11258.00333
Return [%]                           7.712482
Buy & Hold Return [%]              133.269082
Return (Ann.) [%]                    1.507851
Volatility (Ann.) [%]                4.539608
Sharpe Ratio                         0.332154
Sortino Ratio                        0.489443
Calmar Ratio                         0.167841
Max. Drawdown [%]                     -8.9838
Avg. Drawdown [%]                   -0.942717
Max. Drawdown Duration      359 days 00:00:00
Avg. Drawdown Duration       45 days 00:00:00
# Trades                                   10
Win Rate [%]                             90.0
Best Trade [%]                      92.463459
Worst Trade [%]                     -8.229262
Avg. Trade [%]                    

In [3]:
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 0  # Initialize the investment fund

    def init(self):
        close = self.data.Close
        # Calculate RSI
        self.rsi = self.I(ta.RSI, close, timeperiod=14) 
        # Calculate MACD
        macd, signal_line, _ = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)  
        self.macd = self.I(pd.Series, macd)
        self.signal_line = self.I(pd.Series, signal_line)
        self.previous_macd = self.I(pd.Series(macd).shift, 1)
        self.previous_signal_line = self.I(pd.Series(signal_line).shift, 1)
        self.first_mondays = calculate_first_mondays(self.data.index)

    def next(self):
        today = self.data.index[-1]
        if today in self.first_mondays:
            self.fund += self.average_monthly_income_vnd * self.investment_percentage
            print(f"Day: {today} total fund: {self.fund}")

        # Check for buy signal
        if (self.rsi[-1] <= 30):  
            
            share_price = self.data.Close[-1]
            shares_to_buy = self.fund // share_price
            if shares_to_buy > 0:
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy

        #if (self.rsi[-1] < RSI_OVERBOUGHT and
            #self.previous_macd[-1] > self.previous_signal_line[-1] and
            #self.macd[-1] <= self.signal_line[-1]):
            #if self.position:
                #self.position.close()


def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)


    # Calculate first Mondays
    first_mondays = calculate_first_mondays(stock_data.index)


    # Normalize dates for merging
    stock_data.index = stock_data.index.normalize()
    # Diagnostic Check 3: Check entries before merge

    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
    
    # Diagnostic Check 4: Check entries after reindexing but before dropping NA


    # Drop rows with NA values
    stock_data = stock_data.dropna()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    bt.plot(filename=f'{stock_symbol}')
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['VCB', 'VPB', 'FPT', 'REE'] 

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

NameError: name 'Strategy' is not defined

In [10]:
kdc_data = Vnstock().stock(symbol="VPB").quote.history(start='2019-01-01', end='2024-01-04')
kdc_data = kdc_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
kdc_data.set_index('time', inplace=True)
kdc_data.index = pd.to_datetime(kdc_data.index)
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)
# Merge USD/VND data with stock data
kdc_data.index = kdc_data.index.normalize()
kdc_data['usd/vnd'] = usd_vnd_data['Close'].reindex(kdc_data.index) / 1000
kdc_data['Close'] = kdc_data['Close'] / kdc_data['usd/vnd']
kdc_data = kdc_data.dropna()
kdc_data['RSI'] = ta.RSI(kdc_data['Close'], timeperiod=RSI_PERIOD)
# Calculate MACD
kdc_data['MACD'], kdc_data['Signal_Line'], _ = ta.MACD(
    kdc_data['Close'],
    fastperiod=MACD_FAST,
    slowperiod=MACD_SLOW,
    signalperiod=MACD_SIGNAL
)
kdc_data['Previous_MACD'] = kdc_data['MACD'].shift(1)
kdc_data['Previous_Signal_Line'] = kdc_data['Signal_Line'].shift(1)
def macd_rsi_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
        (df['MACD'] >= df['Signal_Line']) &
        (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] < RSI_OVERBOUGHT) &
        (df['Previous_MACD'] > df['Previous_Signal_Line']) &
        (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

kdc_data = macd_rsi_strategy(kdc_data)
kdc_data = kdc_data.drop(columns=['MACD', 'Signal_Line', 'Previous_MACD', 'Previous_Signal_Line',"RSI"])
kdc_data

2024-08-12 08:04:31,399 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,Open,High,Low,Close,Volume,usd/vnd,Signal
time,,,,,,,
2019-01-02,6.69,6.69,6.50,0.280245,1037420,23.194,0
2019-01-03,6.50,6.57,6.27,0.274640,3018830,23.194,0
2019-01-04,6.34,6.42,6.15,0.273778,1928210,23.194,0
2019-01-07,6.47,6.57,6.42,0.278089,1645360,23.194,0
2019-01-08,6.42,6.47,6.35,0.276796,834840,23.194,0
...,...,...,...,...,...,...,...
2023-12-27,17.57,17.66,17.42,0.722302,7676722,24.325,0
2023-12-28,17.57,17.85,17.52,0.733210,10318128,24.345,1
2023-12-29,17.85,18.23,17.76,0.752218,17013562,24.235,0


In [11]:
kdc_data.to_csv("vpb_signal.csv")